<a href="https://colab.research.google.com/github/aaaksenova/NLP-Homework/blob/main/Aksenova_NLP_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Key Words Homework

To-do:

* (1 балл) Подготовить мини-корпус (не меньше 4 текстов, 
примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова. **(DONE)**

* (2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной). **(DONE)**

* (2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf\*idf, OKAPI BM25, ...) **(DONE)**

* (2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.

* (2 балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них. **(DONE)**

* (1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем. **(DONE)**




In [1]:
! pip install spacy --upgrade
! python -m spacy download ru_core_news_md
! pip install python-rake
! pip install summa
! pip install sentence-transformers

     |████████████████████████████████| 6.0 MB 6.4 MB/s 
     |████████████████████████████████| 451 kB 65.8 MB/s 
     |████████████████████████████████| 173 kB 40.7 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 10.1 MB 12.4 MB/s 
     |████████████████████████████████| 628 kB 69.8 MB/s 
  Created wheel for langcodes: filename=langcodes-3.2.1-py3-none-any.whl size=169396 sha256=2dcf4f0dab5106aab9bee985c04cb6ec6857df4e22935a5491fe460a283e0e8e
  Stored in directory: /root/.cache/pip/wheels/12/9c/b3/d42c928e622075d3b6056733125190086e44c9230878e6eb2b
Successfully built langcodes
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing 

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import re
import pandas as pd
import seaborn as sns
import RAKE
from summa import keywords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm_notebook
from tqdm.auto import tqdm
tqdm_notebook().pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


0it [00:00, ?it/s]

## Corpus Creation

Я использую тексты статей из черновика сборника статей по русской грамматике "Язык и метод" 2021. К сожалению, его пока нет в открытом доступе, поэтому ссылку дать не могу. Я выбрала 4 статьи, в т.ч. и свою. В каждой статье окого 3000 токенов и 5-7 ключевых слов.

In [ ]:
with open('key_papers.txt') as fh:
    fil = fh.read(
papers = fil.split('Литература\n')[2:]

In [ ]:
key_to_paper = [(i.split('Ключевые слова: ')[1].split('Key words:')[0], 
                 i.split('Ключевые слова: ')[1].split('Key words:')[1]) for i in papers[:-1]]

In [ ]:
key_words = []
cleaned_papers = []
for key, paper in key_to_paper:
    key = key.strip().strip('.').replace('-\n', '').replace('\n', ' ')
    key_words.append([k for k in key.split(', ')])
    paper = re.sub(r'\[.+?\]', '', paper, flags=re.DOTALL)  # Так убираем ссылки на другие статьи
    paper = re.sub(r'\(.+?\)', '', paper, flags=re.DOTALL)  # Так убираем ссылки на источники примеров
    paper = re.sub(r'[A-Za-z,;1234567890]', '', paper)  # Убираем цифры и английский абстракт с ключевыми словами
    paper = paper.replace('\n', ' ')
    cleaned_papers.append(paper)

In [ ]:
df = pd.DataFrame({'key_words' : [', '.join(key) for key in key_words], 
                   'papers' : cleaned_papers})

In [ ]:
df = df.iloc[16:20]  # Выбираем 4 статьи
df.to_csv('keyword_data.csv', sep='\t')

## Keywords Manual Annotation

Я составляла свою разметку на основе ключевых слов, которые были предложены авторами. Из основных систематических изменений были следующие:

1. Добавлены аббревиатуры. Часто авторы вводят аббревиатуру в начале статьи и дальше используют только ее, а в ключевых словах указана полная форма (совершенный вид - СВ)

2. Все ключевые слова приведены к начальной форме: глаголы –> глагол

3. Некоторые ключевые слова объединяли два концепта, я их разделила (сложное и простое предложение –> сложное предложение, простое предложение)

Также добавила или убрала несколько, но большую часть слов оставила без изменений.

## Automated keyword extraction

In [61]:
df = pd.read_csv('keyword_data.csv', sep='\t') # В этом файле уже исправленные мною ключевые слова
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [62]:
model = spacy.load('ru_core_news_md')


def preprocess(text, model):

    docum = model(text)
    words = ' '.join([w.lemma_ for w in docum if w.lemma_.isalpha()])
    return words

Лемматизируем тексты и ключевые слова, посчитаем сколько ключевых слов у каждого текста

In [63]:
df['prep_texts'] = df.papers.apply(lambda x: preprocess(x, model))
df['prep_kw'] = df.key_words.apply(lambda x: ', '.join([preprocess(i, model) for i in x.split(', ')]))
df['key_count'] = df.key_words.apply(lambda x: len(x.split(', ')))

In [64]:
df

,key_words,papers,prep_texts,prep_kw,key_count
0,"пунктуация, коннектор, союз, аппозитивная конс...",. ’. ...,введение работа посвятить современный особенно...,"пунктуация, коннектор, союз, аппозитивный конс...",10
1,"глагол, св, структура ситуации, широкий контек...",. ....,исследование выполнить при финансовый поддержк...,"глагол, св, структура ситуация, широкий контек...",11
2,"синоним, глагол, пунктуация, синтаксис, орфогр...",. . ...,повтор синоним повтор синоним многогранный явл...,"синоним, глагол, пунктуация, синтаксис, орфогр...",7
3,"предикативный атрибут, па, вторичный предикат,...",. . ...,введение усталый но довольный пионер вернуться...,"предикативный атрибут, па, вторичный предикат,...",6


Я фиксирую число ключевых слов, извлекаемое для каждого текста, чтобы дальше считать метрики классификации выбранным способом.

### Rake

Извлечем ключевые слова с помощью RAKE для лемматизированного и сырого текстов

In [65]:
# Следуя ноутбуку с семинара, добавим слово который и его формы, потомук что spacy иногда его не лемматизирует
stop = stopwords.words('russian') + ['которого',
                                      'которое',
                                      'которой',
                                      'котором',
                                      'которую',
                                      'которые',
                                      'который',
                                      'которых']

In [66]:
rake = RAKE.Rake(stop)

In [67]:
def kw_rake(text, count, rake):
    kw = rake.run(text, maxWords=2, minFrequency=3) # В наших ключевых словах встречаются только униграммы и биграммы
    kw = [k[0] for k in kw]
    return ', '.join(kw[:count])

In [68]:
df['rake_raw'] = df.apply(lambda x: kw_rake(x['papers'], x['key_count'], rake), axis=1)
df['rake_prep'] = df.apply(lambda x: kw_rake(x['prep_texts'], x['key_count'], rake), axis=1)

### TextRank

In [69]:
def kw_textrank(text, count):
    kw = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)[:10]
    return ', '.join([k[0] for k in kw])

In [70]:
df['textrank'] = df.apply(lambda x: kw_textrank(x['prep_texts'], x['key_count']), axis=1)

### Embedding-based

В качестве третьего метода возьмем эмбеддинги. Я нашла код в интернете и заменила модель на LaBSE, потому что она хорошо справляется с русским (судя по моей курсовой работе прошлого года и статьям по пробингу).

Выделим все возможные униграммы и биграммы (кандидаты) с помощью CountVectorizer. Посчитаем косинусную близость между кандидатами и текстом, отсортируем и вынем нужное число кандидатов.

Based on: https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea 

In [71]:
ST = SentenceTransformer('sentence-transformers/LaBSE')

In [72]:
def kw_embeddings(text, model, top_n):
    n_gram_range = (1, 2)
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop).fit([text])
    candidates = count.get_feature_names()
    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return ', '.join(keywords)

In [73]:
df['emb_prep'] = df.progress_apply(lambda x: kw_embeddings(x['prep_texts'], ST, x['key_count']), axis=1)

  0%|          | 0/4 [00:00<?, ?it/s]

In [74]:
df['emb_raw'] = df.progress_apply(lambda x: kw_embeddings(x['papers'], ST, x['key_count']), axis=1)

  0%|          | 0/4 [00:00<?, ?it/s]

## Syntactic patterns for keywords

В основном ключевые слова имеют вид Adj+Noun или Noun+Noun. Давайте будем проверять такие паттерны. Напишем функцию, проверяющую, является ли строка таким паттерном или нет, чтобы отфильтровать датафрейм по паттернам

In [75]:
def check_pattern(text, model):
    docum = model(text)  # Модель у нас все еще spacy nlp
    if len(docum) == 2:
        if docum[1].pos_ == 'NOUN' and docum[0].pos_ in ['NOUN', 'ADJ']:
            return 1
    return 0

## Evaluation metrics

Давайте представим тексты как набор токенов и их сочетаний. Тогда задачу извлечения ключевых слов можно свести к бинарной классификации: каждый элемент набора либо является, либо не является ключевым словом. Выбирая число ключевых слов неравное исходному, мы бы нарушили логику такой классификаци (см выше про фиксированное число кандидатов). Исходя из такой трансформации посчитаем метрики

Я решила считать метрики только по лемматизированным ключевым словам, потому что из сырых текстов извлекались разные формы одного слова (особенно в эмбеддингах)

In [76]:
df_metrics = pd.DataFrame(columns = ['text_id' , 'extraction_algorithm', 'precision' , 'recall', 'f-score'])
df_mask_metrics = pd.DataFrame(columns = ['text_id' , 'extraction_algorithm', 'precision' , 'recall', 'f-score'])


for text_id in tqdm(range(df.shape[0])):

    attributes = {column: df.iloc[text_id][column] 
                  for column in df if 'raw' not in column}
    count = CountVectorizer(ngram_range=(1, 2), 
                            stop_words=stop).fit([attributes['prep_texts']])
    df_prep_tokens = pd.DataFrame({'tokens' : count.get_feature_names()})

    # Считаем матрицу классификации
    for i in ['prep_kw', 'rake_prep', 'textrank', 'emb_prep']:
        df_prep_tokens[i] = df_prep_tokens['tokens'].apply(lambda x: 
                                                           1 if x in attributes[i].split(', ') else 0)
    # Считаем метрики для полных данных
    for i in ['rake_prep', 'textrank', 'emb_prep']:
        precision, recall, fscore, support = precision_recall_fscore_support(df_prep_tokens['prep_kw'], 
                                                                             df_prep_tokens[i], 
                                                                             average='binary', 
                                                                             zero_division=0)
        df_metrics = df_metrics.append(pd.Series((text_id, i, precision, recall, fscore), 
                                                 index=df_metrics.columns), ignore_index=True)
        
    # Считаем метрики для масок
    df_prep_tokens['mask_flag'] = df_prep_tokens.tokens.progress_apply(lambda x: check_pattern(x, model))
    for i in ['rake_prep', 'textrank', 'emb_prep']:
        precision, recall, fscore, support \
         = precision_recall_fscore_support(df_prep_tokens[df_prep_tokens['mask_flag'] == 1]['prep_kw'], 
                                           df_prep_tokens[df_prep_tokens['mask_flag'] == 1][i], 
                                           average='binary', 
                                           zero_division=0)
        df_mask_metrics = df_mask_metrics.append(pd.Series((text_id, i, precision, recall, fscore), 
                                                 index=df_mask_metrics.columns), ignore_index=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2134 [00:00<?, ?it/s]

  0%|          | 0/1994 [00:00<?, ?it/s]

  0%|          | 0/2321 [00:00<?, ?it/s]

  0%|          | 0/2780 [00:00<?, ?it/s]

In [77]:
df_prep_tokens[df_prep_tokens.prep_kw == 1]  # Пример среза по матрице

,tokens,prep_kw,rake_prep,textrank,emb_prep,mask_flag
266,вторичный предикат,1,0,0,0,1
1150,обособление,1,1,1,0,0
1407,па,1,1,1,0,0
1679,предикативный атрибут,1,0,0,0,1
1765,прилагательное,1,0,1,0,0
1928,пунктуация,1,0,0,0,0


**Метрики по всем ключевым словам**

In [78]:
cm = sns.light_palette("purple", as_cmap=True)
pd.pivot_table(df_metrics, index=['text_id', 'extraction_algorithm']).style.background_gradient(cmap=cm)

**Метрики по именным группам**

In [79]:
cm = sns.light_palette("purple", as_cmap=True)
pd.pivot_table(df_mask_metrics, index=['text_id', 'extraction_algorithm']).style.background_gradient(cmap=cm)

## Final discussion

1. В целом все работает очень плохо, а для масок еще хуже. Посмотрим на то, что извлеклось для нулевого текста, где вообще ничего не совпало

In [86]:
df.key_words.loc[1]

'глагол, св, структура ситуации, широкий контекст, накопитель эффекта, предел ситуации, градационный глагол, делимитативный, семельфактивный, способ действия, сд'

In [87]:
df.rake_prep.loc[1]

'глагол св, ситуация, случай, запятые, это'

In [88]:
df.textrank.loc[1]

'глаголы, ситуация, глагол оксана, предела, возможность, также, данном, данный, данному, данного'

In [89]:
df.emb_prep.loc[1]

'контекст возможный, ситуация глагол, глагол возможный, случай возможный, условие объект, ситуация выразить, ситуация объект, ситуация предполагать, концептуализует ситуация, ситуация возможность, ситуация абсолютный'

То есть в принципе извлекаются токены *глагол, св, ситуация* но не в тех комбинациях, в которых они есть в ключевых словах



2. Судя по всему, существуют более адекватные метрики для извлечения ключевых слов и в датасетах, которые сделаны для этой задачи помимо наличия слова в списке учитывается "уверенность" в этом слове, то есть его ранг. Про метрики я читала [тут](https://medium.com/gumgum-tech/exploring-different-keyword-extractors-evaluation-metrics-and-strategies-ef874d336773)

3. Надо как-то получше предобработать данные, возможно, попробовать стемминг, потому что в одной из статей писали об обособлении и смысл выражался разными частями речи (обособлять, обособление, обособленный)

4. Может, нужно иначе выделять ключевые слова руами. Больше смотреть не на смысл, а на частотность и чаще выделять однословные единицы. 

5. Можно было бы сделать кластеризацию эмбеддингов всех кандидатов на ключевые слова и искать близость не отдельных токенов к тексту, а центроидов или просто средних векторов кластеров. А потом брать слово, которое ближе всего к вектору, репрезентующему кластер

6. Вообще эмбеддинги тут хороши, потому что часто авторы статьи выносят в ключевые слова какие-то обобщающие понятия, например, у статьи в журнале может быть тег NLP, но при этом она будет только о NER и сам токен NLP не встретится ни разу. Поэтому, возможно, для метрики качества выделения ключевых слов тоже можно пробовать косинусное расстояние, если мы хотим понять, способен ли наш алгоритм выделять ключевые смыслы (которые по сути упрощаются до ключевых слов). У меня в одной из статей было оригинальное ключевое слово "русский язык", хотя явно в тексте оно не упоминалось.